In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    pred_threshold: 0.5 
    torch_compile: False

log:
    root: '?'
    save_preds: '?' 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: None 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-area_estimation_error'

<IPython.core.display.Javascript object>

In [5]:
%%yaml inference_aug_cfg

inference_augs:
    normalize_min_quantile: 0.98
    normalize_max_scale: 1.02

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [6]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "ISLES_3D_Eval"

base_model: 
    # - "/storage/vbutoi/scratch/ESE/training/09_15_24_ISLES_3D_Dice_wAUG"
    - "/storage/vbutoi/scratch/ESE/calibration/09_16_24_ISLES_3D_LTS_wAug"

inference_data:
    _class: "ese.datasets.ISLES"
    version: 1.0
    split: ("cal", "val")

log:
    save_preds: True

dataloader:
    batch_size: 1

<IPython.core.display.Javascript object>

In [7]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
# base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg, inference_aug_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [8]:
len(inf_cfgs)

3

## Running Jobs

In [9]:
from ese.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/hdd/home/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [10]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[-1], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='0',
) 

In [11]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2']
# )

Submitting job 1/3:
--> Launched job-id: 3525846 on gpu: 0.
Submitting job 2/3:
--> Launched job-id: 3525957 on gpu: 1.
Submitting job 3/3:
--> Launched job-id: 3526105 on gpu: 2.
